<a href="https://colab.research.google.com/github/HesamoddinHosseini/ML-Candidate-Elimination-Algorithm/blob/main/Candidate%20Elimination%20Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
#Candidate Eliminate Algorithm

In [58]:
#Seyed Hesamoddin Hosseini
#Date: November 18 2021

In [59]:
from google.colab import files
uploaded = files.upload()
import pandas as pd
import io
df = pd.read_csv(io.BytesIO(uploaded["DataSet.csv"]))

Saving DataSet.csv to DataSet (1).csv


In [60]:
df

,origin,manufacture,color,decade,type,class
0,japan,honda,blue,1980,economy,+
1,japan,toyota,green,1970,sport,-
2,japan,toyota,blue,1990,economy,+
3,usa,chrysler,red,1980,economy,-
4,japan,honda,white,1980,economy,+
5,japan,toyota,green,1980,economy,+


In [61]:
def get_domains(examples):
    d = [set() for i in examples[0]]
    for x in examples:
        for i, xi in enumerate(x):
            d[i].add(xi)
    return [list(sorted(x)) for x in d]



In [62]:
get_domains(examples)

[['japan', 'origin', 'usa'],
 ['chrysler', 'honda', 'manufacture', 'toyota'],
 ['blue', 'color', 'green', 'red', 'white'],
 ['1970', '1980', '1990', 'decade'],
 ['economy', 'sport', 'type'],
 ['+', '-', 'class']]

In [63]:
import numpy as np
attribute =np.array(df)[0:3,:-1]
target=np.array(df)[0:3,:-1]

test_attribute=np.array(df)[3:6,:-1]
test_target=np.array(df)[3:6,-1]


In [64]:
print(attribute)

[['japan' 'honda' 'blue' 1980 'economy']
 ['japan' 'toyota' 'green' 1970 'sport']
 ['japan' 'toyota' 'blue' 1990 'economy']]


In [65]:
print(test_attribute)

[['usa' 'chrysler' 'red' 1980 'economy']
 ['japan' 'honda' 'white' 1980 'economy']
 ['japan' 'toyota' 'green' 1980 'economy']]


In [66]:

range(len(target))


range(0, 3)

In [71]:
num_attribute=len(attribute[0])
specific_hypothesis=[0]*num_attribute


In [12]:
specific_hypothesis

[0, 0, 0, 0, 0]

In [13]:
num_attribute=len(attribute[0])
general_hypothesis=['?']*num_attribute

In [33]:
general_hypothesis

['?', '?', '?', '?', '?']

In [67]:
def g_0(n):
    return ("?",)*n

def s_0(n):
    return ('0',)*n  

In [68]:
def more_general(h1, h2):
    more_general_parts = []
    for x, y in zip(h1, h2):
        mg = x == "?" or (x != "0" and (x == y or y == "0"))
        more_general_parts.append(mg)
    return all(more_general_parts)


In [69]:
# min_generalizations
def fulfills(attribute, hypothesis):
    ### the implementation is the same as for hypotheses:
    return more_general(hypothesis, attribute)

def min_generalizations(h, x):
    h_new = list(h)
    for i in range(len(h)):
        if not fulfills(x[i:i+1], h[i:i+1]):
            h_new[i] = '?' if h[i] != '0' else x[i]
    return [tuple(h_new)]

In [70]:
def min_specializations(h, domains, x):
    results = []
    for i in range(len(h)):
        if h[i] == "?":
            for val in domains[i]:
                if x[i] != val:
                    h_new = h[:i] + (val,) + h[i+1:]
                    results.append(h_new)
        elif h[i] != "0":
            h_new = h[:i] + ('0',) + h[i+1:]
            results.append(h_new)
    return results

In [73]:
def candidate_elimination(attribute):
    domains = get_domains(attribute)[:-1]
    
    G = set([g_0(len(domains))])
    S = set([s_0(len(domains))])
    i=0
    print("\n G[{0}]:".format(i),G)
    print("\n S[{0}]:".format(i),S)
    for xcx in attribute:
        i=i+1
        x, cx = xcx[:-1], xcx[-1]  
        if cx=='Y': # x is positive example
            G = {g for g in G if fulfills(x, g)}
            S = generalize_S(x, G, S)
        else: # x is negative example
            S = {s for s in S if not fulfills(x, s)}
            G = specialize_G(x, domains, G, S)
        print("\n G[{0}]:".format(i),G)
        print("\n S[{0}]:".format(i),S)
    return 

In [74]:
def generalize_S(x, G, S):
    S_prev = list(S)
    for s in S_prev:
        if s not in S:
            continue
        if not fulfills(x, s):
            S.remove(s)
            Splus = min_generalizations(s, x)
            ## keep only generalizations that have a counterpart in G
            S.update([h for h in Splus if any([more_general(g,h) 
                                               for g in G])])
            ## remove hypotheses less specific than any other in S
            S.difference_update([h for h in S if 
                                 any([more_general(h, h1) 
                                      for h1 in S if h != h1])])
    return S

In [75]:
def specialize_G(x, domains, G, S):
    G_prev = list(G)
    for g in G_prev:
        if g not in G:
            continue
        if fulfills(x, g):
            G.remove(g)
            Gminus = min_specializations(g, domains, x)
            ## keep only specializations that have a conuterpart in S
            G.update([h for h in Gminus if any([more_general(h, s)
                                                for s in S])])
            ## remove hypotheses less general than any other in G
            G.difference_update([h for h in G if 
                                 any([more_general(g1, h) 
                                      for g1 in G if h != g1])])
    return G

In [76]:
candidate_elimination(attribute)


 G[0]: {('?', '?', '?', '?')}

 S[0]: {('0', '0', '0', '0')}

 G[1]: {('?', '?', '?', 1970), ('?', '?', '?', 1990), ('?', 'toyota', '?', '?'), ('?', '?', 'green', '?')}

 S[1]: {('0', '0', '0', '0')}

 G[2]: {('?', 'honda', '?', 1970), ('?', '?', '?', 1990), ('?', '?', 'blue', 1970), ('?', 'toyota', 'blue', '?'), ('?', '?', 'green', 1980), ('?', 'honda', 'green', '?'), ('?', 'toyota', '?', 1980)}

 S[2]: {('0', '0', '0', '0')}

 G[3]: {('?', 'honda', '?', 1970), ('?', 'honda', '?', 1990), ('?', '?', 'blue', 1970), ('?', '?', 'green', 1990), ('?', '?', 'green', 1980), ('?', 'honda', 'green', '?'), ('?', 'toyota', '?', 1980)}

 S[3]: {('0', '0', '0', '0')}


In [77]:
candidate_elimination(test_attribute)


 G[0]: {('?', '?', '?', '?')}

 S[0]: {('0', '0', '0', '0')}

 G[1]: {('?', 'honda', '?', '?'), ('?', '?', 'white', '?'), ('?', 'toyota', '?', '?'), ('?', '?', 'green', '?'), ('japan', '?', '?', '?')}

 S[1]: {('0', '0', '0', '0')}

 G[2]: {('japan', 'chrysler', '?', '?'), ('usa', 'honda', '?', '?'), ('usa', '?', 'white', '?'), ('?', 'chrysler', 'white', '?'), ('?', 'honda', 'red', '?'), ('?', 'toyota', '?', '?'), ('japan', '?', 'red', '?'), ('?', '?', 'green', '?')}

 S[2]: {('0', '0', '0', '0')}

 G[3]: {('japan', 'chrysler', '?', '?'), ('usa', 'honda', '?', '?'), ('?', 'toyota', 'red', '?'), ('usa', '?', 'white', '?'), ('?', 'chrysler', 'green', '?'), ('?', 'chrysler', 'white', '?'), ('usa', '?', 'green', '?'), ('?', 'toyota', 'white', '?'), ('?', 'honda', 'red', '?'), ('japan', '?', 'red', '?'), ('usa', 'toyota', '?', '?'), ('?', 'honda', 'green', '?')}

 S[3]: {('0', '0', '0', '0')}
